### **Representing text as numbers**
<br>
Machine learning models take vectors (arrays of numbers) as input.<br> When working with text, the first thing you must do is come up with a strategy to convert strings to numbers (or to "vectorize" the text) before feeding it to the model.<br> In this section, you will look at three strategies for doing so.

### **One-hot encodings**
<br>
As a first idea, you might "one-hot" encode each word in your vocabulary. Consider the sentence "The cat sat on the mat". <br>The vocabulary (or unique words) in this sentence is (cat, mat, on, sat, the). <br>To represent each word, you will create a zero vector with length equal to the vocabulary, then place a one in the index that corresponds to the word.

In [1]:
import io
import os
import re
import shutil
import string
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

**Download the IMDb Dataset**

In [2]:
# Download the dataset using Keras file utility and take a look at the directories.

url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz", url,
                                  untar=True, cache_dir='.',
                                  cache_subdir='')

dataset_dir = os.path.join(os.path.dirname(dataset), 'aclImdb')
os.listdir(dataset_dir)

84140032/84125825 [==============================] - 8s 0us/step


['imdbEr.txt', 'train', 'test', 'imdb.vocab', 'README']

In [3]:
train_dir = os.path.join(dataset_dir, 'train')
os.listdir(train_dir)

['labeledBow.feat',
 'unsupBow.feat',
 'pos',
 'neg',
 'urls_unsup.txt',
 'urls_neg.txt',
 'unsup',
 'urls_pos.txt']

In [4]:
# The train directory also has additional folders which should be removed before creating training dataset.

remove_dir = os.path.join(train_dir, 'unsup')
shutil.rmtree(remove_dir)

In [6]:
# Use the train directory to create both train and validation datasets with a split of 20% for validation.

batch_size = 1024
seed = 123
train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='training', seed=seed)
val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size, validation_split=0.2,
    subset='validation', seed=seed)

<BatchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [7]:
# Take a look at a few movie reviews and their labels (1: positive, 0: negative) from the train dataset.

for text_batch, label_batch in train_ds.take(1):
  for i in range(5):
    print(label_batch[i].numpy(), text_batch.numpy()[i])

0 b"Oh My God! Please, for the love of all that is holy, Do Not Watch This Movie! It it 82 minutes of my life I will never get back. Sure, I could have stopped watching half way through. But I thought it might get better. It Didn't. Anyone who actually enjoyed this movie is one seriously sick and twisted individual. No wonder us Australians/New Zealanders have a terrible reputation when it comes to making movies. Everything about this movie is horrible, from the acting to the editing. I don't even normally write reviews on here, but in this case I'll make an exception. I only wish someone had of warned me before I hired this catastrophe"
1 b'This movie is SOOOO funny!!! The acting is WONDERFUL, the Ramones are sexy, the jokes are subtle, and the plot is just what every high schooler dreams of doing to his/her school. I absolutely loved the soundtrack as well as the carefully placed cynicism. If you like monty python, You will love this film. This movie is a tad bit "grease"esk (without

**Configure the dataset for performance**



In [8]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

**Using the Embedding layer**

In [9]:
# Embed a 1,000 word vocabulary into 5 dimensions.
embedding_layer = tf.keras.layers.Embedding(1000, 5)

In [10]:
# If you pass an integer to an embedding layer, the result replaces each integer with the vector from the embedding table:

result = embedding_layer(tf.constant([1, 2, 3]))
result.numpy()

array([[ 0.03418522, -0.04443232,  0.01137059,  0.03934163, -0.00701485],
       [ 0.01581841,  0.00361831, -0.03961027,  0.02432462, -0.03556474],
       [-0.01867376, -0.01374419,  0.01390532,  0.04119274,  0.03310594]],
      dtype=float32)

In [ ]:
# The returned tensor has one more axis than the input, the embedding vectors are aligned along the new last axis. Pass it a (2, 3) input batch and the output is (2, 3, N)

result = embedding_layer(tf.constant([[0, 1, 2], [3, 4, 5]]))
result.shape

TensorShape([2, 3, 5])

### **Text preprocessing**

In [ ]:
## Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')


# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 100

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

### **Create a classification model**

In [ ]:
embedding_dim=16

model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(1)
])

### **Compile and train the model**

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [ ]:
# Compile and train the model using the Adam optimizer and BinaryCrossentropy loss.

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[tensorboard_callback])

Epoch 1/15
20/20 [==============================] - 5s 229ms/step - loss: 0.6920 - accuracy: 0.5028 - val_loss: 0.6897 - val_accuracy: 0.4886
Epoch 2/15
20/20 [==============================] - 2s 91ms/step - loss: 0.6864 - accuracy: 0.5028 - val_loss: 0.6824 - val_accuracy: 0.4886
Epoch 3/15
20/20 [==============================] - 2s 91ms/step - loss: 0.6769 - accuracy: 0.5028 - val_loss: 0.6708 - val_accuracy: 0.4886
Epoch 4/15
20/20 [==============================] - 2s 92ms/step - loss: 0.6623 - accuracy: 0.5028 - val_loss: 0.6542 - val_accuracy: 0.4886
Epoch 5/15
20/20 [==============================] - 2s 93ms/step - loss: 0.6422 - accuracy: 0.5060 - val_loss: 0.6330 - val_accuracy: 0.5048
Epoch 6/15
20/20 [==============================] - 2s 91ms/step - loss: 0.6169 - accuracy: 0.5571 - val_loss: 0.6077 - val_accuracy: 0.5728
Epoch 7/15
20/20 [==============================] - 2s 93ms/step - loss: 0.5871 - accuracy: 0.6324 - val_loss: 0.5798 - val_accuracy: 0.6366
Epoch 8/15
2

In [ ]:
# You can look into the model summary to learn more about each layer of the model.

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_vectorization (TextVect (None, 100)               0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 16)           160000    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 160,289
Trainable params: 160,289
Non-trainable params: 0
_________________________________________________________________


In [ ]:
!%load_ext tensorboard
!%tensorboard --logdir logs

/bin/bash: line 0: fg: no job control
/bin/bash: line 0: fg: no job control


### **Retrieve the trained word embeddings and save them to disk**

In [ ]:
#  The get_vocabulary() function provides the vocabulary to build a metadata file with one token per line.

weights = model.get_layer('embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

In [ ]:
# Write the weights to disk. To use the Embedding Projector, you will upload two files in tab separated format: a file of vectors (containing the embedding), 
# and a file of meta data (containing the words).

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [ ]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>